In [ ]:
#pip install pyautogui

In [1]:
import cv2
import tensorflow.keras
import numpy as np
import mediapipe as mp
import time
import pyautogui # 활성화된 창 조절
import tkinter as tk # win 전체 스크린 화면크기 확인
import threading, requests, time
#import pyautogui as pag

In [21]:
## 이미지 전처리
size=0
name=""
OnOff=False
conversion_T = 0 # 일정간격으로 휠체어 판별
Change_S = True # 사이즈 변경 여부
Detect_E = False # 눈 감지 여부
IsRatio = True # 비율 조정 여부
IsNonWheel_Count = 0
IsWheel_Count = 0
result = 0

def preprocessing(frame):
    # 사이즈 조정
    size = (224, 224)
    frame_resized = cv2.resize(frame, size, interpolation=cv2.INTER_AREA)
    
    # 이미지 정규화
    frame_normalized = (frame_resized.astype(np.float32) / 127.0) - 1
    
    # 이미지 차원 재조정 - 예측을 위해 reshape 해줍니다.
    frame_reshaped = frame_normalized.reshape((1, 224, 224, 3))
    
    return frame_reshaped


def Is_Run():
    ## 학습된 모델 불러오기
    model_filename = 'C:/Users/Inwook/Desktop/학습/4학년1학기/캡스톤/ELK/ELK_통합/keras_model.h5'
    model = tensorflow.keras.models.load_model(model_filename)

    # # mediapipe 초기값 설정
    mp_drawing = mp.solutions.drawing_utils
    mp_face_detection = mp.solutions.face_detection
    face_detection = mp_face_detection.FaceDetection(
        min_detection_confidence=0.7)

    # 카메라 캡쳐 객체, 0=내장 카메라
    capture = cv2.VideoCapture(0)

    # 캡쳐 프레임 사이즈 조절
    capture.set(cv2.CAP_PROP_FRAME_WIDTH, 520)
    capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 520)

    global conversion_T # 일정간격으로 휠체어 판별
    global Change_S # 사이즈 변경 여부
    global Detect_E # 눈 감지 여부
    global IsRatio # 비율 조정 여부
    global IsNonWheel_Count
    global IsWheel_Count
    global result

    w = pyautogui.getWindowsWithTitle(name)[0] # win프로그램 권한받기

    win = tk.Tk() # screen 크기 확인하기 (win 중앙값 찾기)
    screen_width = win.winfo_screenwidth()
    screen_height = win.winfo_screenheight()
    win.destroy() # tk 중지

    center_P = 0 # 얼굴 중앙 좌표 값

    w.resizeTo(screen_width,screen_height)

    while OnOff: # 전체 프로그램 반복 실행 

    #    while Change_S: # 사이즈 변경 모듈
    #         result = pyautogui.prompt("사이즈 값 입력","입력") # 값 받기

    #         if result == "":
    #             continue

    #         size = int(result)
    #         w.resizeTo(size,size)
    #         w.moveTo(0,0)

    #         check = pyautogui.confirm("현사이즈로 하시겠습니까?","확인")
    #         if check == "OK":
    #             Change_S = False # 사이즈 변경 모듈 실행 중지
    #             w.resizeTo(screen_width,screen_height)

    #             break
        


        WaitingForNext = True # 눈인식 모듈 무한 실행

        ret, frame = capture.read()
        if not ret: 
            print("error")

        # 이미지 뒤집기
        frame_fliped = cv2.flip(frame, 1)

        # 얼굴 좌표값 얻기
        results = face_detection.process(cv2.cvtColor(frame_fliped, cv2.COLOR_BGR2RGB))
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(frame_fliped, detection)

                x1 = detection.location_data.relative_bounding_box.xmin # left side of face bounding box
                x2 = x1 + detection.location_data.relative_bounding_box.width # right side of face bounding box
                y1 = detection.location_data.relative_bounding_box.ymin
                y2 = y1 + detection.location_data.relative_bounding_box.height

                center_PX = (x1 + x2) / 2 # center of the face
                center_PY = (y1 + y2) / 2 

                break    

        # 이미지 출력
        cv2.imshow("VideoFrame", frame_fliped)

        # 1초마다 검사하며, videoframe 창으로 아무 키나 누르게 되면 종료
        if cv2.waitKey(1) == ord('q'): 
            break

        # 데이터 전처리
        preprocessed = preprocessing(frame_fliped)

        # 예측
        prediction = model.predict(preprocessed)
        #print(prediction) # [[0.00533728 0.99466264]]

        conversion_T += 1 # 대기 시간

        # 휠체어 탑승여부 출력
        if conversion_T % 30 == 0:
            print(conversion_T, Detect_E)

            # wheelchair(0), normal(1)
            if prediction[0,0] < prediction[0,1]:
                print('휠체어를 안타고 있다.')

                Detect_E = False
                IsNonWheel_Count += 1
                IsWheel_Count = 0
                if IsNonWheel_Count == 3:
                    IsRatio = True #비율 조정 가능
                    w.resizeTo(screen_width,screen_height)
                    w.moveTo(0,0)
                    
                    #w.maximize()
                    IsNonWheel_Count = 0
            else:
                print('휠체어를 타고 있다')
                IsNonWheel_Count = 0
                Detect_E = True
                IsWheel_Count += 1

        if(Detect_E == True) and (IsWheel_Count == 3): # 눈 인식 모듈

            if IsRatio: 
                IsRatio = False #비율 조정 불가능
                WaitingForNext = False
                print(WaitingForNext)
               
                w.resizeTo(size,size)
                w.centerx = screen_width*center_PX
                w.centery = screen_height*center_PY
                w.activate()


        if conversion_T == 90: # 대기 시간이 0~90 사이를 반복하게 함
            conversion_T = 0 # 대기 시간 초기화
            Detect_E = False # 눈 인식 판별 초기화


    # 카메라 객체 반환
    capture.release() 
    # 화면에 나타난 윈도우들을 종료
    cv2.destroyAllWindows()
    


def setOption():
    global size
    global name
    size = int(entry2.get())
    name = entry1.get()
    print(name)
    print(size)
    print("임창민바보")


def On_C():
    global OnOff
    OnOff = True
    print("바보")
    t1 = threading.Thread(target=Is_Run)
    t1.start()
      


def Off_C():
    global OnOff
    OnOff = False
    print("OnOff")
    t1.stop()

root = tk.Tk()
root.title("ELK")
root.geometry("600x400")

upper_frame = tk.Frame(root, width=400, height=70)
upper_frame.pack(pady=20)
 
down_frame = tk.Frame(root, width=400, height=100)
down_frame.pack(padx=10, pady=10)

label=tk.Label(upper_frame, text="프로그램 이름", width=16, fg="red",font = 'sans 10 bold')
label.grid(column=0, row=0, padx=10, pady=5)

entry1 = tk.Entry(upper_frame, width=15, font=("Courier",18), borderwidth=5)
entry1.grid(column=1, row=0, padx=10, pady=5)

label3=tk.Label(upper_frame, text="*정확한 프로그램 이름을 입력해주세요", width=30, fg="red",font = 'sans 8 bold')
label3.grid(column=1, row=1, padx=10, pady=5)

#빈 라벨
label4=tk.Label(upper_frame, text="", width=30, fg="red",font = 'sans 8 bold')
label4.grid(column=1, row=2, padx=10, pady=5)


label2=tk.Label(upper_frame, text="프로그램 설정 크기", width=16, fg="red",font = 'sans 10 bold')
label2.grid(column=0, row=3 , padx=10, pady=5)

entry2 = tk.Entry(upper_frame, width=15, font=("Courier",18), borderwidth=5)
entry2.grid(column=1, row=3, padx=10, pady=5)

#빈 라벨
label5=tk.Label(upper_frame, text="*숫자만 입력해주세요", width=30, fg="red",font = 'sans 8 bold')
label5.grid(column=1, row=4, padx=10, pady=5)


btn7 = tk.Button(down_frame,text='O N', padx=5, font = 'sans 16 bold', width=3, command= On_C)
btn7.grid(column=0, row=5, padx=40, pady=5)


btn8 = tk.Button(down_frame,text='OFF', padx=5, font = 'sans 16 bold',width=3,command= Off_C)
btn8.grid(column=1, row=5,padx=40, pady=5)

btn설정 = tk.Button(down_frame,text='설정', padx=5, font = 'sans 16 bold',width=3, command = setOption)
btn설정.grid(column=2, row=5, padx=40, pady=5)

root.mainloop() 
  

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Inwook\anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-21-09a6892981e2>", line 178, in setOption
    size = int(entry2.get())
ValueError: invalid literal for int() with base 10: ''
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Inwook\anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-21-09a6892981e2>", line 178, in setOption
    size = int(entry2.get())
ValueError: invalid literal for int() with base 10: ''
